In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = "retina"
%load_ext autoreload
%autoreload 2
import importlib.resources
from pyslime import slime
from pyslime import utils as pu

import pandas as pd
import numpy as np
from importlib import reload
from matplotlib import pyplot as plt
import glob
import pickle
from astropy.table import Table, vstack, unique
from astropy.cosmology import Planck15 as cosmo

from pyslime.pipeline import catalogs, interpolate


datadir = "/Volumes/GoogleDrive/My Drive/SlimeMold/2021-11-23-VACv2/"
packagedir = "/Users/mwilde/python/pyslime/pyslime/data/"
dropboxdir = "/Users/mwilde/Dropbox/slime-mold/data/final_data/"
pipedatadir = "/Users/mwilde/python/pyslime/pyslime/pipeline/data/"


In [ ]:
glob.glob(datadir+"*")

# this pretab is made via 
`catalogs.make_galaxy_cat_with_slimedens(datadir, packagedir)`

In [ ]:
# pretab = catalogs.make_galaxy_cat_with_slimedens(datadir, pipedatadir)
pretab = Table.read(datadir+"prevac_catalog.csv", format='ascii.csv')

In [ ]:
xbreak0, y_const0, pfit0, xbreak5, y_const5, pfit5 = catalogs.load_mapfuncs(pipedatadir)

In [ ]:
bins = np.linspace(-1,4, 500)


z0 = interpolate.relu_p3(bins, xbreak0, y_const0, pfit0)
z5 = interpolate.relu_p3(bins, xbreak5, y_const5, pfit5)

plt.plot(bins, z0, label='z=0.0')
plt.plot(bins, z5, label='z=0.5')
plt.xlabel('sm dens')
plt.ylabel('bp dens')
plt.legend();

In [ ]:
bins = np.linspace(-1,4, 1000)
kwargs = {}
kwargs['bins'] = bins
kwargs['histtype'] = 'step'
single_slice = pretab[pretab['MCPM_RUN'] == 7]

plt.hist(single_slice['log10_slimedens'], **kwargs, label='sm');
plt.hist(single_slice['log10_overdens'], **kwargs, label='mapped overdensity');
# plt.hist(mapfunc0(pretab['log10_slimedens']), **kwargs, label='map0');
# plt.hist(mapfunc5(pretab['log10_slimedens']), **kwargs, label='map0.5');
plt.legend(bbox_to_anchor=(1.05,1))
plt.title("dens at galaxy loc");
plt.xlabel("dens")


In [ ]:
bins = np.linspace(-1,4, 1000)
kwargs = {}
kwargs['bins'] = bins
kwargs['histtype'] = 'step'
pretab = pretab[pretab['MCPM_RUN'] == 1]

z0 = interpolate.relu_p3(pretab['log10_slimedens'], xbreak0, y_const0, pfit0)
z5 = interpolate.relu_p3(pretab['log10_slimedens'], xbreak5, y_const5, pfit5)

g = np.random.normal(2.8, 0.3, 100000)

zg = interpolate.relu_p3(g, xbreak0, y_const0, pfit0)

plt.hist(pretab['log10_slimedens'], **kwargs, label='sm');
# plt.hist(pretab['log10_overdens'], **kwargs, label='rho');
plt.hist(z0, **kwargs, label='map0');
# plt.hist(x(pretab['log10_slimedens']), **kwargs, label='map0');
plt.hist(g, **kwargs, label='gaussian')
plt.hist(zg, **kwargs, label='gaussian')
# plt.hist(mapfunc5(pretab['log10_slimedens']), **kwargs, label='map0.5');
plt.legend(bbox_to_anchor=(1.05,1))
plt.title("dens at galaxy loc");
plt.xlabel("dens")
# plt.xlim(1.45, 1.58)


In [ ]:
df = pretab.to_pandas()

In [ ]:
galcat = Table.read(datadir + "galaxy_cat_forSlime.csv", format="ascii.ecsv")

In [ ]:
galcat = galcat.to_pandas()

In [ ]:
spike = df.loc[(df.log10_overdens > 1.4) & (df.log10_overdens < 1.6)]

In [ ]:
plt.plot(spike.log10_slimedens, spike.log10_overdens, '.')
# plt.plot(df.log10_slimedens, df.log10_overdens, '.')

In [ ]:
print(len(df))
df.nunique()

In [ ]:
print(len(spike))
spike.nunique()

In [ ]:
spike.log10_slimedens.min(), spike.log10_slimedens.max()

# Here are the duplicated values in slimedens

In [ ]:
def get_dups(df, col):
    dups = pd.concat(g for _, g in df.groupby(col) if len(g) > 1)
    return dups

get_dups(df, "log10_slimedens")

In [ ]:
slimedir = '/Volumes/GoogleDrive/My Drive/SlimeMold/2021-11-23-VACv2/LRG_NGC_z=0-1000mpc'

slimeobj = slime.Slime.get_slime(slimedir, standardize=True)

In [ ]:
tab = galcat
lumcut = [0, 1000]
flag = 1 

if "NGC" in slimeobj.name:
    subtable = tab[tab["survey"] == "LRG_NGC"]
elif "SGC" in slimeobj.name:
    subtable = tab[tab["survey"] == "LRG_SGC"]
elif "SDSS" in slimeobj.name:
    subtable = tab[tab["survey"] == "SDSS"]
else:
    print("there is no subtable, chck this!!!!")

t = pd.DataFrame()
t["ra"] = subtable["RA"]
t["dec"] = subtable["DEC"]
t["z"] = subtable["Z"]
t["mstars"] = subtable["MassStars"]
t["lumdist"] = subtable["lumDist"]
t["MCPM_RUN"] = flag

cut = t[(t["lumdist"] > lumcut[0]) & (t["lumdist"] < lumcut[1])]
galtrace = slimeobj.sky_to_data(
    cut["ra"], cut["dec"], cut["z"], check_bounds=True)

In [ ]:
galtrace.shape

In [ ]:
rho = mapfunc0(galtrace)

In [ ]:
u, c = np.unique(galtrace, return_counts=True)
dup = u[c > 1 ]
plt.hist(dup, bins=50);

In [ ]:
u, c = np.unique(rho, return_counts=True)
dup = u[c > 1 ]
plt.hist(dup, bins=50);

In [ ]:
df_trace = pd.Series({"galtrace":galtrace})

In [ ]:
df_trace.duplicated()

In [ ]:
cut

In [ ]:
get_dups(df_trace, "galtrace")